# Cvičenie 12: Strojové učenie v Pythone

Na poslednom cvičení v tomto semestri sa pozrieme na niekoľko základných knižníc, ktoré sa často používajú pri implementácii inteligentných riešení v Pythone, a s ktorými sa určite stretnete aj na ďalších predmetoch. Ukážeme si jeden príklad trénovania klasifikačného modelu na jednoduchých dátach pomocou strojového učenia, a dostanete priestor na tvorbu vlastných modelov.

V priebehu cvičenia budeme používať hneď niekoľko knižníc, ktoré by mali byť súčasťou Anaconda distribúcie. Ak počas vypracovania cvičenia vám niektoré knižnice budú chýbať, môžete si ich doinštalovať pomocou príkazu `pip`:

```
pip install numpy
pip install pandas
pip install seaborn
pip install matplotlib
pip install scikit-learn
```

Postup vývoja klasifikačných modelov vieme rozdeliť do nasledujúcich krokov:

1. predspracovanie údajov
2. návrh modelu
3. trénovanie modelu
4. vyhodnotenie modelu

## 1. Predspracovanie údajov

Predspracovanie údajov je prvý krok, ktorý v sebe zahŕňa hneď niekoľko úloh:
* načítanie datasetu
* výber príznakov
* normalizácia hodnôt
* vektorizácia vstupov a výstupov
* rozdelenie datasetu na trénovaciu, testovaciu a validačnú množinu.

Postup pri predspracovaní údajov ukážeme na [Iris datasete](sources/lab12/iris.csv). Dataset popisuje niekoľko kvetov pomocou ich základných rozmerov.

### 1.1. Načítanie datasetu

Na načítanie datasetu existujú rôzne knižnice pre Python, jedna populárna z nich je knižnica `pandas`. Knižnica dokáže načítať rôzne formátované dáta, napríklad formáty csv, html, json, hdf5 a SQL. Náš dataset vieme načítať priamo zo súboru csv nasledovným spôsobom:

In [ ]:
import pandas as pd
dataset = pd.read_csv('iris.csv')
print(dataset)

Načítaný dataset má typ DataFrame. K ľubovoľným stĺpcom sa dostaneme zadaním názvu stĺpca ako index datasetu. Ak chceme zobraziť viac stĺpcov, index musí byť zoznam s názvami týchto stĺpcov.

In [ ]:
# select only column SepalLengthCm
print(dataset['SepalLengthCm'])

# select columns SepalLengthCm and SepalWidthCm
print(dataset[['SepalLengthCm', 'SepalWidthCm']])

Alternatívne vieme zobraziť stĺpce ako keby boli parametrom objektu dataset, alebo vieme použiť aj poradové číslo stĺpca (znak `:` pred čiarkou vyjadruje všetky riadky).

In [ ]:
print(dataset.SepalLengthCm)
print(dataset.iloc[:, 0])

Indexovanie riadkov a stĺpcov viete aj kombinovať, na poradí nezáleží:

In [ ]:
print(dataset[:10]['SepalLengthCm'])
print(dataset['SepalLengthCm'][:10])

Z datasetu viete vybrať iba niektoré riadky aj na základe hodnoty niektorého atribútu použitím `lambda` funkcie. Napríklad, pre všetky riadky, kde hodnota SepalLengthCm je viac ako 5:

In [ ]:
print(dataset.loc[lambda df:df.SepalLengthCm > 5, :])

Všetky tieto podmnožiny majú typ `DataFrame`. Ak chcete hodnoty použiť ako zoznam, resp. zoznam zoznamov, musíte pridať `values`:

In [ ]:
dataset['SepalLengthCm'][:10:].values

### 1.2. Výber príznakov

Pred výberom príznakov potrebujeme získať intuitívne pochopenie datasetu a vzťahov medzi jednotlivými atribútmi a výsledkom klasifikácie. V tomto nám pomôže knižnica `Seaborn`, ktorá slúži na vizualizáciu údajov a využíva knižnicu `matplotlib`.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# set plot style
sns.set(style="ticks")
sns.set_palette("husl")

# create plots over all dataset; for subset use iloc indexing
sns.pairplot(dataset, hue="Species")

# display plots using matplotlib
plt.show()

Uvedený kód namapuje záznamy z datasetu v každom možnom príznakovom priestore vo dvojiciach príznakov.

Z grafov vidíme, že ani jedna kombinácia nám nedá lineárne separovateľný dataset, budeme teda používať všetky príznaky, ktoré vyberieme pomocou knižnice `pandas`:

In [ ]:
# split data into input (X - select the first four columns) and output (y - select last column)
X = dataset.iloc[:, :4].values
y = dataset.iloc[:, -1].values

### 1.3. Normalizácia hodnôt

Normalizácia hodnôt sa používa najmä pre zložitejšie datasety a urýchli proces trénovania modelov, ktoré lepšie pracujú s dátami z istého intervalu. Počas normalizácie sa číselné hodnoty namapujú zvyčajne na interval 0 až 1. Vzhľadom na jednoduchosť našich dát tento krok nateraz môžeme vynechať.

### 1.4. Vektorizácia vstupov a výstupov

Kým väčšina klasifikačných modelov dokáže spracovať iba číselné hodnoty, skoro všetky datasety obsahujú aj nečíselné údaje (reťazce, kategórie, booleovské hodnoty, atď.). Preto je potrebné, aby sme tieto hodnoty premenili na vektorovú reprezentáciu. Pri vektorizácii upravíme výstupy na formu *n* čísel, kde *n* je počet tried pri klasifikácii. Každý vektor bude obsahovať práve jednu 1 a ostatné hodnoty budú 0, tieto čísla vyjadrujú mieru príslušnosti k jednotlivým triedam.

V našom datasete potrebujeme upraviť očakávaný výstup, ktorý zatiaľ má formu reťazca. Pri vektorizácii vieme využiť `LabelEncoder` z knižnice `scikit-learn`:

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
# transform string labels into number values 0, 1, 2
y1 = encoder.fit_transform(y)

# transform number values into vector representation
Y = pd.get_dummies(y1).values

**Poznámka:** v niektorých prípadoch dokážete reťazce nahradiť jednoduchými číslami, takýto spôsob ale predpokladá, že čísla, ktoré sú blízko sebe vyjadrujú koncepty, ktoré sú veľmi podobné. Napríklad, ak máme stĺpec s hodnotami low, middle, high, tieto hodnoty vieme nahradiť číslami 1, 2 a 3. Rovnaký spôsob ale nemôžeme použiť s hodnotami ako napríklad značky auta: Škoda (1), Audi (2), Lada (3), pretože neurónová sieť by predpokladala, že Lada (3) je viac podobná Audi (2) ako Škodovke (1).

### 1.5. Rozdelenie datasetu na trénovaciu, testovaciu a validačnú množinu

Ďalšou úlohou je rozdelenie množiny na trénovaciu a testovaciu. Na to použijeme ďalšiu funkciu z knižnice `scikit-learn`, a to `train_test_split` ([dokumentácia](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)), ktorá zachová poradie vstupov a výstupov a má tri dôležité parametre:
1. zoznam vstupov
2. zoznam výstupov
3. test_size - veľkosť testovacej množiny medzi 0 a 1 (môžete použiť aj train_size)

Pre opakovateľnosť trénovania je odporúčané používať random seed zadaním parametra `random_state`.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

Validačná množina sa pri jednoduchých datasetoch až tak často nepoužíva, slúži ako testovacia množina počas fázy trénovania a môže byť použitá ako podmienka pre ukončenie trénovania. Keď ju chcete získať, môžete tak urobiť opätovným volaním `train_test_split` nad už vyselektovanou trénovacou množinou.

## 2. Návrh modelu

Knižnica `scikit-learn` ponúka implementáciu skoro každého modelu kontrolovaného učenia, [ich zoznam nájdete tu](https://scikit-learn.org/stable/supervised_learning.html). Tieto modely sa naučia správne klasifikovať zadané príklady do tried na základe príkladov, s ktorými sa stretnú počas trénovania. Každý model je definovaný niekoľkými parametrami, ktoré potrebujete vhodne nastaviť, aby ste dosiahli čo najpresnejšiu predikciu.

Pre jednoduchosť pri našom prvom pokuse použijeme klasifikátor [*k najbližších susedov*](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification), ktorý predpovedá triedu vzorky na základe väčšiny tried jej *k* najbližších susedov v trénovacích dátach. Je jednoduchý, neparametrický a dobre funguje pri menších datasetoch, kde rozhodovanie závisí na lokálnej podobnosti medzi vzorkami. Môžeme ho zadefinovať jednoducho vytvorením objektu:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()

## 3. Trénovanie modelu

Ak sme spokojní s naším modelom, môžeme ho začať trénovať pomocou fukcie `fit`. V tomto kroku potrebujeme zadať trénovacie príklady: vstupy aj výstupy, na základe ktorých sa model naučí základné charakteristiky dát.

In [ ]:
knn.fit(X_train, y_train)

## 4. Vyhodnotenie modelu

Vyhodnotenie siete pozostáva z dvoch základných úloh: testovanie a vyhodnotenie. Pre testovanie musíme získať výstupy ku vstupným hodnotám z trénovacej množiny pomocou metódy `predict`:

In [ ]:
y_pred = knn.predict(X_test)

Ďalej porovnáme ozajstné výstupy s očakávanými. Keďže výstup má vektorovú reprezentáciu, potrebujeme zistiť pozíciu kde sa nachádza najväčšia hodnota vo vektore. V tomto nám pomôže knižnica `numpy`.

Pre vyhodnotenie našej siete použijeme konfúznu maticu. Konfúzna matica je tabuľková reprezentácia, kde v riadkoch máme očakávané triedy a v stĺpcoch vypočítané (predikované). V bunkách tabuľky sú uložené počty príkladov klasifikované v danej kombinácii očakávanej a predikovanej triedy. Ideálny klasifikátor bude mať všetky hodnoty po hlavnej diagonále (ďalšie informácie nájdete na [wikipédii](https://en.wikipedia.org/wiki/Confusion_matrix)).

In [ ]:
import numpy as np

y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test_class, y_pred_class))

Z konfúznej matici potom vieme vypočítať ďalšie metriky, ako správnosť (*accuracy*), návratnosť (*recall*) a presnosť (*precision*):

In [ ]:
print(classification_report(y_test_class, y_pred_class))

Správnosť popisuje samotný klasifikátor a vypočíta sa nasledovne:

$ACC = \frac{TP + TN}{P + N}$

kde TP + TN je suma správne klasifikovaných príkladov (na hlavnej diagonále) a P + N je počet všetkých príkladov.

Návratnosť a presnosť popisujú klasifikátor pre danú triedu, vypočítajú sa nasledovne:

$REC = \frac{TP}{P}$

$PREC = \frac{TP}{TP + FP}$

kde TP je počet správne klasifikovaných príkladov z danej triedy, P je počet príkadov z danej triedy v testovacej množine a FP je počet príkladov z testovacej množiny nesprávne klasifikovaných do tejto triedy.

Metóda `classification_report` vypočíta ešte hodnotu F1, ktorá je harmonický priemer návratnosti a presnosti:

$F1 = 2 \cdot \frac{REC \cdot PREC}{REC + PREC}$

## Doplnkové úlohy
1. Vyskúšajte ďalšie klasifikačné modely zo `scikit-learn`u, a porovnajte ich presnosti.
2. Pri použití parametrických klasifikátorov je kľúčové správne nastavenie parametrov, čo sa najčastejšie rieši veľkým počtom menších pokusov s rôznymi nastaveniami parametrov a ich vyhodnotením. Na konci procesu sa vyberú parametre, ktoré dajú najlepšiu presnosť. Vyskúšajte tento proces na vybranom klasifikačnom modeli, inšpirovať sa môžete [oficiálnym tutoriálom](https://scikit-learn.org/1.4/tutorial/statistical_inference/model_selection.html).
3. Skúste natrénovať ľubovoľný model na inom štandardnom datasete. Ďalšie datasety nájdete priamo v knižnici `scikit-learn` [v module `datasets`](https://scikit-learn.org/stable/datasets/toy_dataset.html).